In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import mne
from pathlib import Path
import regex as re
from functools import reduce


In [2]:
def get_all_set(directory):
  """
  Gets a list of all .set files in a directory and its subdirectories.

  Args:
    directory: The path to the directory.

  Returns:
    A list of file paths.
  """
  directory = Path(directory)

  # Search recursively for .set files
  set_files = list(directory.rglob("*.set"))
  return set_files

In [3]:
eeg_data = mne.io.read_raw_eeglab((os.path.join('EEG_human', 'ds004504-1.0.8', 'sub-001', 'eeg', 'sub-001_task-eyesclosed_eeg.set')), preload=True)
# Convert to Pandas DataFrame
df = eeg_data.to_data_frame()
df.set_index('time', inplace=True)
df.columns = 'sub-001-' + df.columns
df = df[df.columns[:3]]

In [4]:
sub_meta = pd.read_table(os.path.join('EEG_human', 'ds004504-1.0.8','participants.tsv'))
all_eeg_recordings = get_all_set(os.path.join('EEG_human', 'ds004504-1.0.8', 'derivatives'))
all_control_dfs = []
all_ad_dfs = []
all_ftd_dfs = []
for fp in all_eeg_recordings:
    try:
        temp_eeg_data = mne.io.read_raw_eeglab(fp, preload=True)
        temp_df = temp_eeg_data.to_data_frame()
        temp_df.set_index('time', inplace=True)
        # Get subject ID
        subject = re.findall(r'sub-\d{3}', str(fp))[0]
        # Get subject group (C, A, F)
        subject_group = sub_meta[sub_meta['participant_id'] == subject].iloc[0]['Group']
        # Rename columns to contain subject ID
        temp_df.columns = f'{subject}-' + temp_df.columns
        # Keep only first three recording channels
        temp_df = temp_df[temp_df.columns[:3]]
        if subject_group == 'C':
            all_control_dfs.append(temp_df)
        if subject_group == 'A':
            all_ad_dfs.append(temp_df)
        elif subject_group == 'F':
            all_ftd_dfs.append(temp_df)
    except Exception as e:
        print(f'Error: {e}')
all_control_subjects = reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True, how='outer'), all_control_dfs)
all_ad_subjects = reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True, how='outer'), all_ad_dfs)
all_ftd_subjects = reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True, how='outer'), all_ftd_dfs)
all_control_subjects.to_csv(os.path.join('EEG_human', 'control_eeg_all.csv'))
all_ad_subjects.to_csv(os.path.join('EEG_human', 'ad_eeg_all.csv'))
all_ftd_subjects.to_csv(os.path.join('EEG_human', 'ftd_eeg_all.csv'))

C:\Users\garve\AppData\Local\Temp\ipykernel_23480\527154934.py:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  temp_eeg_data = mne.io.read_raw_eeglab(fp, preload=True)
C:\Users\garve\AppData\Local\Temp\ipykernel_23480\527154934.py:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  temp_eeg_data = mne.io.read_raw_eeglab(fp, preload=True)
C:\Users\garve\AppData\Local\Temp\ipykernel_23480\527154934.py:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  temp_eeg_data = mne.io.read_raw_eeglab(fp, preload=True)
C:\Users\garve\AppData\Local\Temp\ipykernel_23480\527154934.py:8: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching 